In [1]:
#Reference Paper:
'''
[1]LINGO, an Efficient Holographic Text Based Method To Calculate Biophysical
Properties and Intermolecular Similarities
David Vidal, Michael Thormann ,and Miquel Pons
Journal of Chemical infomation and modeling
2005,45(2),pp 386-393
http://pubs.acs.org/doi/abs/10.1021/ci0496797
[2]Lingos, finite state machines, and fast similarity searching
J. Andrew Grant, James A. Haigh, Barry T. Pickup et al.
2006,46(5),pp 1912-1918
http://pubs.acs.org/doi/abs/10.1021/ci6002152
'''

def similar(mols,q):
    #get lingos:
    'q: the length of the lingo'
    lingos = []
    for mol in mols:
        n = len(mol)
        for i in range(n-q+1):
            if mol[i:i+q] in lingos:
                continue
            lingos.append(mol[i:i+q])
    #calculate Tc
    l = len(lingos)
    numerator = 0
    'cnumerator = 0 '
    for lingo in lingos:
        NA = sum([mols[0][i:i+q]==lingo for i in range(len(mols[0])-q+1)])
        NB = sum([mols[1][i:i+q]==lingo for i in range(len(mols[1])-q+1)])
        numerator += 1 - abs(NA-NB) / float(NA+NB)
        'cnumerator += min(NA,NB)'
    Tc = numerator / float(l) #Reference [1]
    'Tab = float(cnumerator) / (len(mols[0])+len(mols[1]) - 2*q +2 -cnumerator)' #Reference[2]
    return Tc #Or Tab


#Test
mols = ['CC(C)C=CCCCCC(=O)NCc1ccc(c(c1)OC)O','COC1=C(C=CC(=C1)C=O)O']

print(similar(mols,4))

0.044444444444444446


In [3]:
ABCB1['Canonical_SMILES'][0] = ABCB1['Canonical_SMILES'][0].replace('<br />','')


In [4]:
print(ABCB1['Canonical_SMILES'][0])



CC(C)C(=O)OC1=C(C=C(C=C1)CO)C(CCN(C(C)C)C(C)C)C2=CC=CC=C2


In [24]:
import pandas as pd

#read data from csv as 'molList'
molList = pd.read_csv('data/ABCB1.csv',index_col=0)
molList.head()



,TransporterID,Gene_name,Protein_Name_generally,Synonyms,DrugID,Drug_Name,synonyms,Canonical_SMILES
0,DTD0003,ABCB1,P-glycoprotein 1,ABC20; ABCB1; ATP-binding cassette sub-family ...,DR00001,Fesoterodine fumarate,Toviaz (TN),CC(C)C(=O)OC1=C(C=C(C=C1)CO)C(CCN(C(C)C)C(C)C)...
1,DTD0003,ABCB1,P-glycoprotein 1,ABC20; ABCB1; ATP-binding cassette sub-family ...,DR00002,Paraquat,"1,1'-dimethyl-4,4' bipyridinium dichloride",C[N+]1=CC=C(C=C1)C2=CC=[N+](C=C2)C.[Cl-].[Cl-]
2,DTD0003,ABCB1,P-glycoprotein 1,ABC20; ABCB1; ATP-binding cassette sub-family ...,DR00008,Venetoclax,Venclexta,CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...
3,DTD0003,ABCB1,P-glycoprotein 1,ABC20; ABCB1; ATP-binding cassette sub-family ...,DR00013,Nilotinib,Nilotinib; Tasigna; AMN107; AMN 107; AMN-107; ...,CC1=C(C=C(C=C1)C(=O)NC2=CC(=CC(=C2)C(F)(F)F)N3...
4,DTD0003,ABCB1,P-glycoprotein 1,ABC20; ABCB1; ATP-binding cassette sub-family ...,DR00014,Simeprevir,TMC436,CC1=C(C=CC2=C1N=C(C=C2OC3CC4C(C3)C(=O)N(CCCCC=...


In [36]:
results = pd.DataFrame(columns=['Drug', 'Score','GeneName'])

In [37]:
#create for loop to calculate the similarity between molX and each mol in ABCB1 and store the results in the array 'results'
molX = 'CCCCCC(C=CC1C(CC(=O)C1CC=CCCCC(=O)O)O)O'
for i in range(0,len(molList['Canonical_SMILES'])):
    results.loc[i] = [molList['Drug_Name'][i], similar([molX,molList['Canonical_SMILES'][i]],4),molList['Gene_name'][i]] 


#results.head()
display(results)

,Drug,Score,GeneName
0,Fesoterodine fumarate,0.114583,ABCB1
1,Paraquat,0.045198,ABCB1
2,Venetoclax,0.046032,ABCB1
3,Nilotinib,0.058333,ABCB1
4,Simeprevir,0.060564,ABCB1
...,...,...,...
428,Tenofovir disoproxil,0.051741,ABCB1
429,S-licarbazepine,0.100709,ABCB1
430,Saphris,0.126984,ABCB1
431,BMS-3870032,0.054645,ABCB1


In [38]:
#binarize the results with threshold 0.6
results['Similarity'] = results['Score'].apply(lambda x: 1 if x>0.6 else 0)
display(results['Similarity'].sum())

results[results['Similarity']==1]


1

,Drug,Score,GeneName,Similarity
348,Prostaglandin A2,0.719048,ABCB1,1


In [21]:
ABCB1['Canonical_SMILES'][0]
molCorrect = 'CC(C)C(=O)OC1=C(C=C(C=C1)CO)C(CCN(C(C)C)C(C)C)C2=CC=CC=C2.C(=CC(=O)O)C(=O)O'
#save the similarity results versis DrugID to data frame 'results'
score = similar([molList['Canonical_SMILES'][0],molCorrect],4)


score

0.7614379084967321